# Создание чат-бота

Чатбот на 2 интента:
- Простая болталка
- Вопрос-ответ в качестве доврачебной медицинской консультации


In [1]:
! pip install -q transformers

In [2]:
import numpy as np
import pandas as pd
import re
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification, AutoModelForCausalLM
import tensorflow as tf
import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive/')
path =  "/content/drive/My Drive/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Загрузим токенайзер и модель классификатора интентов:

In [4]:
tokenizer_intent_classifier = AutoTokenizer.from_pretrained(path + 'tokenizer_intent_classifier/')
model_intent_classifier = TFAutoModelForSequenceClassification.from_pretrained(path + 'model_intent_classifier/')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/My Drive/model_intent_classifier/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Загрузим токенайзер и модель GPT для "болталки":

In [5]:
device = torch.device("cpu")

In [6]:
tokenizer_dialog = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
model_dialog = AutoModelForCausalLM.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2').to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Загрузим токенайзер и модель GPT для вопросно-ответной системы:

In [7]:
tokenizer_qa_medical_gen = AutoTokenizer.from_pretrained(path + 'tokenizer_qa_medical_gen')
model_qa_medical_gen = AutoModelForCausalLM.from_pretrained(path + 'model_qa_medical_gen').to(device)

Напишем класс для диалоговой системы DialogSystem.

**Основные методы объекта класса DialogSystem:**
- intent_classifier - классифицирует входящее сообщение на 3 интента. Если текущий интент меняется, обнуляет историю сообщений.
- dialog_generator - генерирует ответы для двух интентов - болталки и медицинской вопросно-ответной системы. При разных интентах на вход подаются разные GPT-модели
- search_article - поиск 3-х наиболее подходящих отзывов в ответ на сообщение. Для 3-го интента
- Объединяет все эти методы метод get_response. Именно он и будет вызываться как внешняя функция.
- Также в классе есть вспомогательные функции для выполнения принципа DRY (Don't Repeat Yourself)


In [8]:
class DialogSystem:


    def __init__(self,
                 tokenizer_intent_classifier,
                 model_intent_classifier,
                 tokenizer_dialog,
                 model_dialog,
                 tokenizer_qa_medical_gen,
                 model_qa_medical_gen,
                 lenght_history = 10):
        self.context = [] # Список, хранящий lenght_history предыдущих сообщений
        self.current_intent = 0 # Текущий интент. При изменении интента обнуляется self.context
        self.tokenizer_intent_classifier = tokenizer_intent_classifier
        self.model_intent_classifier = model_intent_classifier
        self.tokenizer_dialog = tokenizer_dialog
        self.model_dialog = model_dialog
        self.tokenizer_qa_medical_gen = tokenizer_qa_medical_gen
        self.model_qa_medical_gen = model_qa_medical_gen
        self.lenght_history = lenght_history


    def intent_classifier(self, sentence):
        sent_token = self.tokenizer_intent_classifier(sentence, padding="max_length", truncation=True, return_tensors='tf')
        predict_intent = self.model_intent_classifier(sent_token).logits.numpy().argmax()
        if predict_intent != self.current_intent:
            self.context = []
        self.current_intent = predict_intent
        print(f'current intent: {predict_intent}')
        return predict_intent


    def custom_append(self, a, b):
        a.append(b)
        if len(a) > self.lenght_history:
            a.pop(0)


    def respond_to_dialog(self, model, tokenizer, texts):
        prefix = '\nx:'
        for i, t in enumerate(texts):
            prefix += t
            prefix += '\nx:' if i % 2 == 1 else '\ny:'
        tokens = tokenizer(prefix, return_tensors='pt')
        tokens = {k: v.to(model.device) for k, v in tokens.items()}
        end_token_id = tokenizer.encode('\n')[0]
        size = tokens['input_ids'].shape[1]
        output = model.generate(
            **tokens,
            eos_token_id=end_token_id,
            do_sample=True,
            max_length=size+128,
            repetition_penalty=3.2,
            temperature=1,
            num_beams=3,
            length_penalty=0.01,
            pad_token_id=tokenizer.eos_token_id
        )
        decoded = tokenizer.decode(output[0])
        result = re.findall(r'\ny:(.+)', decoded)[-1]
        return result.strip()


    def dialog_generator(self, model, tokenizer, sentence):
        self.custom_append(self.context, sentence)
        result = self.respond_to_dialog(model, tokenizer, self.context)
        self.custom_append(self.context, result)
        return result

    def get_response(self, sentence):
        predict_intent = self.intent_classifier(sentence)
        if predict_intent == 0:
            res = self.dialog_generator(self.model_dialog, self.tokenizer_dialog, sentence)
        else:
            res = self.dialog_generator(self.model_qa_medical_gen, self.tokenizer_qa_medical_gen, sentence)
        return res


Создадим объект responser класса DialogSystem:

In [9]:
responser = DialogSystem(tokenizer_intent_classifier,
                         model_intent_classifier,
                         tokenizer_dialog,
                         model_dialog,
                         tokenizer_qa_medical_gen,
                         model_qa_medical_gen)

Создадим тестовую функцию, аналогичную callback-функции в MessageHandler, чтобы протестировать работу нашей диалоговой системы, не подключаясь к api telegram:

In [10]:
def test_text_message():
    message = input()
    answer = responser.get_response(message)
    print(answer)

Создадим бесконечный цикл с вызовом тестовой функции:

In [15]:
while True:
    test_text_message()

Привет. Как дела?
current intent: 0
Все отлично.
Погода хорошая, как думаешь?
current intent: 1
Здравствуйте. У меня такой же вопрос; Здравствуйте! Мне 28 лет, у меня есть проблемы с почками и суставами, в основном это остеохондроз позвоночника, грыжа межреберного сустава поясничного сплетения, протрузии на коленных суставах, артроз нижних тазобедренных суставов, кистозная болезнь, атеросклероз сосудов шейного отдела позвоночника, ревматоидный артрит грудного отдела позвоночника, хронический панкреатит, нарушение работы поджелудочной железы, гастрит, холецистит, дисбактериоз желчного пузыря, гепатит
Что посоветуешь принимать, если болит голова второй день?
current intent: 1
Анализ мочи, общий анализ крови, УЗИ органов брюшной полости, мочевого пузыря, почек, мочеиспускания, биохимия крови, кровь на гормоны щитовидной железы, хеликобактер пилори, эндокринные заболевания, амилазу, эритроциты, иммуноглобулин Е, лейкоциты, креатинин, тестостерон, кальций, магний, железо, медь, гематокрит, 

KeyboardInterrupt: Interrupted by user

## Создание telegram-bot


In [11]:
!pip install python-telegram-bot==13.13

In [12]:
from telegram import Update

In [15]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [16]:
updater = Updater('6735041502:AAFXosAjDAMiuMPrK279xTLaaXentvsDm-w') # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update: Update, context: CallbackContext):
    text = 'Вас приветствует бот MedAQ!\nПообщайтесь или задайте вопрос по беспокоящим Вас симптомам.'
    update.message.reply_text(text)

def textMessage(update: Update, context: CallbackContext):
    input_txt = update.message.text
    answer = responser.get_response(input_txt)
    update.message.reply_text(answer)

start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

<ipython-input-16-7a8079a55ed9>:17: TelegramDeprecationWarning: The argument `clean` of `start_polling` is deprecated. Please use `drop_pending_updates` instead.
  updater.start_polling(clean=True)


current intent: 0
current intent: 0
current intent: 1
current intent: 1
current intent: 0
current intent: 0
